# 📝 Multilingual Text Summarization (French + English)

## 📘 Context

Text summarization is a crucial NLP task used to extract key insights from long documents. With the advancement of transformer-based architectures like BART and T5, we can now generate high-quality summaries in different languages.

This notebook demonstrates how to perform automatic summarization in:
- 🇬🇧 **English**, using `facebook/bart-large-cnn`
- 🇫🇷 **French**, using `plguillou/t5-base-fr-sum-cnndm`

## 🎯 Objectives

- Load and compare language-specific summarization models
- Generate and display summaries for both English and French input texts
- Test edge cases and observe model behavior

## Packages

In [1]:
# !pip install transformers sentencepiece
!pip install -r requirements.txt

     -------------------------------------- 212.4/212.4 MB 1.5 MB/s eta 0:00:00
     ------------------------------------- 981.5/981.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 18.1/18.1 MB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 45.8/45.8 kB ? eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 14.9 MB/s eta 0:00:00
     -------------------------------------- 443.9/443.9 kB 6.9 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 45.3/45.3 kB ? eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.3 MB/s eta 0:00:00
     -------------------------------------- 134.5/134.5 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
     -----------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.18.1 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
conda-repo-cli 1.0.24 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.24 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.24 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.4.0 which is incompatible.


In [2]:
# import loguru

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import textwrap  # Text wrapping and filling

import gradio as gr
from langdetect import detect